In [1]:
from scipy.stats import norm
import scipy.linalg as linalg
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
from datetime import datetime
import math
import matplotlib.pyplot as plt

# Problem 1

## Finite Difference Class

In [3]:
class FiniteDifferences(object):
    
    def __init__(self,S0,K,r,T,sigma,M,N,is_call=True):
                
        
        self.S0=S0
        self.K=K
        self.r=r
        self.T=T
        self.sigma=sigma       
        #self.Smax=Smax
        self.M=int(M)
        self.N=int(N)
        self.Smax=self.S0*np.exp(self.N* 1/(self.M+2))
        Smax=self.S0*np.exp(self.N* 1/(self.M+2))
        self.is_call= is_call       
        self.dS=Smax/float(self.M) # Dividing S into M number of intervals
        self.dt=T/float(self.N)   # Dividing T into N number of intervals
        self.i_values=np.arange(self.M)
        self.j_values=np.arange(self.N)
        self.grid=np.zeros(shape=(self.M+1,self.N+1))  # Constructing the grid 
        self.boundary_conds= np.linspace(0,Smax,self.M+1)
        
    def _setup_boundary_conditions_(self):
        
        pass
    
    
    def _setup_coefficients_(self):
        
        pass
    
    
    
    def _traverse_grid_(self):
        
        pass
    
    
    
    def _interpolate_(self):
        
        
        return np.interp(self.S0, self.boundary_conds, self.grid[:,0])
    
    
    def price(self):
        
        
        self._setup_boundary_conditions_()
        
        self._setup_coefficients_()
        
        self._traverse_grid_()
        
        
        return self._interpolate_()


### FDExplicitEu class

In [6]:

class FDExplicitEu(FiniteDifferences):
    
    
    def _setup_boundary_conditions_(self):
        
        
        if self.is_call:
            
            self.grid[:,-1]= np.maximum(
                self.boundary_conds-self.K,0)
            
            
            self.grid[-1, :-1]= (self.Smax - self.K) * \
                                np.exp(-self.r *
                                       
                                       self.dt *
                                       
                                       (self.N-self.j_values))
            
    
        else:
            self.grid[:, -1] = \
                np.maximum(self.K-self.boundary_conds, 0)
            self.grid[0, :-1] = (self.K - self.Smax) * \
                                np.exp(-self.r *
                                        self.dt *
                                        (self.N-self.j_values))
            

    def _setup_coefficients_(self):
        self.a = 0.5*self.dt*((self.sigma**2) *
                            (self.i_values**2) -
                            self.r*self.i_values)
        self.b = 1 - self.dt*((self.sigma**2) *
                              
                              (self.i_values**2) +
                                self.r)
        self.c = 0.5*self.dt*((self.sigma**2) *
                            (self.i_values**2) +
                            self.r*self.i_values)
        
 
    def _traverse_grid_(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i,j] = self.a[i]*self.grid[i-1,j+1] +\
                                self.b[i]*self.grid[i,j+1] + \
                                self.c[i]*self.grid[i+1,j+1]


#### European Call Option price using explicit difference method

In [8]:
#### European Call Option price using explicit difference method

option = FDExplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=True)

print(option.price())

17.811600777587547


#### European Put Option price using explicit difference method

In [9]:
option = FDExplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=False)

print(option.price())

0.14529840919604145


### FDImplicitEu class

In [10]:
class FDImplicitEu(FDExplicitEu):
    def _setup_coefficients_(self):
        self.a = 0.5*(self.r*self.dt*self.i_values -
            (self.sigma**2)*self.dt*(self.i_values**2))
        self.b = 1 + \
            (self.sigma**2)*self.dt*(self.i_values**2) + \
            self.r*self.dt
        self.c = -0.5*(self.r * self.dt*self.i_values +
            (self.sigma**2)*self.dt*(self.i_values**2))
        self.coeffs = np.diag(self.a[2:self.M], -1) + \
            np.diag(self.b[1:self.M]) + \
            np.diag(self.c[1:self.M-1], 1)
        


    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.coeffs)
        aux = np.zeros(self.M-1)
        
        for j in reversed(range(self.N)):
            aux[0] = np.dot(-self.a[1], self.grid[0, j])
            x1 = linalg.solve(L, self.grid[1:self.M, j+1]+aux)
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2


#### European Call Option price using implicit difference method

In [11]:
option= FDImplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=True)

print (option.price())

11.542325827594295


#### European Put Option price using implicit difference method

In [12]:
option= FDImplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=False)

print (option.price())

13.26916724241089


### Crank-Nicolson Finite Difference method

In [13]:
class FDCnEu(FDExplicitEu):
    def _setup_coefficients_(self):
        self.alpha = 0.25*self.dt*(
        (self.sigma**2)*(self.i_values**2) -
        self.r*self.i_values)
        self.beta = -self.dt*0.5*(
        (self.sigma**2)*(self.i_values**2) +
        self.r)
        self.gamma = 0.25*self.dt*(
        (self.sigma**2)*(self.i_values**2) +
        self.r*self.i_values)
        self.M1 = -np.diag(self.alpha[2:self.M], -1) + \
        np.diag(1-self.beta[1:self.M]) - \
        np.diag(self.gamma[1:self.M-1], 1)
        self.M2 = np.diag(self.alpha[2:self.M], -1) + \
        np.diag(1+self.beta[1:self.M]) + \
        np.diag(self.gamma[1:self.M-1], 1)
    def _traverse_grid_(self):
        """  We are solving the linear systems of equations """
        P, L, U = linalg.lu(self.M1)
        for j in reversed(range(self.N)):
            x1 = linalg.solve(L,
            np.dot(self.M2,
            self.grid[1:self.M, j+1]))
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2

#### European Call Option price using Crank-Nicolson method

In [14]:
option = FDCnEu(S0=50, K=50, r=0.1, T=1, sigma=0.4, M=3, N=3, is_call=True)

print(option.price())

3.6597266500731678


#### European Put Option price using Crank-Nicolson method

In [15]:
option = FDCnEu(S0=50, K=50, r=0.1, T=1, sigma=0.4, M=3, N=3, is_call=False)

print(option.price())

7.143375995657307


### Question e

Parameters: S0 = 100;K = 100, T = 1 year, sigma = 20%; r = 6%; div = 2%

#### Explicit EUCall

In [16]:
option = FDExplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=True)

print(option.price())

17.811600777587547


#### Explicit EUPut

In [17]:
option = FDExplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=False)

print(option.price())

0.14529840919604145


#### Implicit EUCall

In [18]:
option= FDImplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=True)

print (option.price())

11.542325827594295


#### Implicit EUPut

In [19]:
option= FDImplicitEu(S0=100, K=100, r=0.06, T=1, sigma=0.2, M=3, N=3, is_call=False)

print (option.price())

13.26916724241089


#### Crank-Nicolson EUCall

In [20]:
option = FDCnEu(S0=100, K=100, r=0.1, T=1, sigma=0.2, M=3, N=3, is_call=True)

print(option.price())

11.129968769236118


#### Crank-Nicolson EUPut

In [21]:
option = FDCnEu(S0=100, K=100, r=0.1, T=1, sigma=0.2,M=3, N=3, is_call=False)

print(option.price())

11.643816394845299


### Question f

In [22]:
## Blackscholes function to calulate call option price

# S= Stock Price

# K= Strike Price

# t= Expiration Date

# sig= Volatility

# optype= Type

# r= risk free interest rate



def blackscholes_C(S,K,t,sig,r=0.0008):
    
    d1= (np.log(S/K)+(r+sig**2/2)*t)/(sig*np.sqrt(t))
    
    d2= d1-sig*np.sqrt(t)
        
    call_price=norm.cdf(d1,0,1)*S-  norm.cdf(d2,0,1)*K*np.exp(-r*t)
        
    return call_price

In [23]:
blackscholes_C(S=100,K=100,t=1,sig=0.20,r=0.06)

10.98954915262599

# Problem 2

## Importing and Organising Data

In [24]:
# Importing option chain from yahoo finance, and organizing the dataframe  

def get_optionchain(inpt,exprdt):
    
# expiration date format should be like this "2020-03-12"
    
       
    stock=yf.Ticker(inpt)
    
    opt=stock.option_chain(exprdt)
    
    call=opt.calls
    
    put=opt.puts
    
    option_chain=call.append(put)
    
    
    
    
    a=option_chain.drop(["lastTradeDate","change","percentChange","volume","openInterest","inTheMoney","contractSize","currency"],axis=1)
    
    a["Expiration Date"]=exprdt
    
    
    a.columns=['Option Name', 'Strike',"Last Price","Bid","Ask","Implied Volatility","Expiration Date"]
    
    a.reset_index(drop=True,inplace=True)
    
    
    
    # Loop to assign P or C values depending on the type of the option
    for i,j in a.iterrows():
        
        if j["Option Name"][-9]=="P":
                
       
            a.loc[i,"Type"]="put"
        
        
        
        
        elif j["Option Name"][-9]=="C":
                
        
            a.loc[i,"Type"]="call"
        
              
    
    
    a = a[['Option Name',"Expiration Date","Type",'Strike',"Bid","Ask","Last Price","Implied Volatility"]]
    
    a.sort_values(by=['Strike'], inplace=True, ascending=True)

    
    
    return a


In [25]:
# example for the function above


a1=get_optionchain("AMZN",exprdt="2021-03-26")
a2=get_optionchain("AMZN",exprdt="2021-04-16")
a3=get_optionchain("AMZN",exprdt="2021-05-21")


AMZN_opt1=a1.append(a2).append(a3)

AMZN_opt1=AMZN_opt1.reset_index(drop=True)

AMZN_opt1


,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility
0,AMZN210326P01660000,2021-03-26,put,1660.0,0.0,0.0,0.01,0.500005
1,AMZN210326P01680000,2021-03-26,put,1680.0,0.0,0.0,0.03,0.500005
2,AMZN210326P01690000,2021-03-26,put,1690.0,0.0,0.0,0.05,0.500005
3,AMZN210326C01700000,2021-03-26,call,1700.0,1432.5,1442.5,1363.05,3.289553
4,AMZN210326P01700000,2021-03-26,put,1700.0,0.0,0.0,0.03,0.500005
...,...,...,...,...,...,...,...,...
1133,AMZN210521C04700000,2021-05-21,call,4700.0,0.0,0.0,1.78,0.125009
1134,AMZN210521C04800000,2021-05-21,call,4800.0,0.0,0.0,1.60,0.250007
1135,AMZN210521C04900000,2021-05-21,call,4900.0,0.0,0.0,1.33,0.250007
1136,AMZN210521C05000000,2021-05-21,call,5000.0,0.0,0.0,1.25,0.250007


In [26]:
# Subsetting only call options

AMZN_calls=AMZN_opt1.loc[AMZN_opt1["Type"]=="call"].reset_index(drop=True)
AMZN_calls

,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility
0,AMZN210326C01700000,2021-03-26,call,1700.0,1432.5,1442.5,1363.05,3.289553
1,AMZN210326C01710000,2021-03-26,call,1710.0,0.0,0.0,1335.65,0.000010
2,AMZN210326C01730000,2021-03-26,call,1730.0,0.0,0.0,1315.70,0.000010
3,AMZN210326C01740000,2021-03-26,call,1740.0,0.0,0.0,1343.95,0.000010
4,AMZN210326C01760000,2021-03-26,call,1760.0,0.0,0.0,1285.75,0.000010
...,...,...,...,...,...,...,...,...
558,AMZN210521C04600000,2021-05-21,call,4600.0,0.0,0.0,2.01,0.125009
559,AMZN210521C04700000,2021-05-21,call,4700.0,0.0,0.0,1.78,0.125009
560,AMZN210521C04800000,2021-05-21,call,4800.0,0.0,0.0,1.60,0.250007
561,AMZN210521C04900000,2021-05-21,call,4900.0,0.0,0.0,1.33,0.250007


In [27]:
# Subsetting only put options

AMZN_puts=AMZN_opt1.loc[AMZN_opt1["Type"]=="put"].reset_index(drop=True)
AMZN_puts

,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility
0,AMZN210326P01660000,2021-03-26,put,1660.0,0.0,0.00,0.01,0.500005
1,AMZN210326P01680000,2021-03-26,put,1680.0,0.0,0.00,0.03,0.500005
2,AMZN210326P01690000,2021-03-26,put,1690.0,0.0,0.00,0.05,0.500005
3,AMZN210326P01700000,2021-03-26,put,1700.0,0.0,0.00,0.03,0.500005
4,AMZN210326P01710000,2021-03-26,put,1710.0,0.0,0.28,0.22,2.187505
...,...,...,...,...,...,...,...,...
570,AMZN210521P04400000,2021-05-21,put,4400.0,0.0,0.00,1338.05,0.000010
571,AMZN210521P04500000,2021-05-21,put,4500.0,0.0,0.00,1447.10,0.000010
572,AMZN210521P04600000,2021-05-21,put,4600.0,0.0,0.00,1437.28,0.000010
573,AMZN210521P04700000,2021-05-21,put,4700.0,0.0,0.00,1575.75,0.000010


Subsettin AMZN at the money calls for 3 different expiration date

In [28]:
# AMZN at the money calls for 3 different expiration date
AMZN_ATM_calls=AMZN_calls[(AMZN_calls["Strike"]>1700) & (AMZN_calls["Strike"]<2000)].reset_index(drop=True)
#AMZN_ATM_calls

AMZN_ATM_calls=AMZN_ATM_calls.sort_values("Strike",ascending=True).reset_index(drop=True)
AMZN_ATM_calls

,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility
0,AMZN210326C01710000,2021-03-26,call,1710.0,0.00,0.00,1335.65,0.000010
1,AMZN210416C01710000,2021-04-16,call,1710.0,0.00,0.00,1366.30,0.000010
2,AMZN210416C01720000,2021-04-16,call,1720.0,0.00,0.00,1386.00,0.000010
3,AMZN210326C01730000,2021-03-26,call,1730.0,0.00,0.00,1315.70,0.000010
4,AMZN210416C01730000,2021-04-16,call,1730.0,0.00,0.00,1384.70,0.000010
5,AMZN210326C01740000,2021-03-26,call,1740.0,0.00,0.00,1343.95,0.000010
6,AMZN210416C01750000,2021-04-16,call,1750.0,0.00,0.00,1354.80,0.000010
7,AMZN210326C01760000,2021-03-26,call,1760.0,0.00,0.00,1285.75,0.000010
8,AMZN210416C01760000,2021-04-16,call,1760.0,0.00,0.00,1500.75,0.000010
9,AMZN210326C01770000,2021-03-26,call,1770.0,0.00,0.00,1314.00,0.000010


Subsettin AMZN at the money put for 3 different expiration date

In [29]:
# AMZN at the money puts for 3 different expiration date
AMZN_ATM_puts=AMZN_puts[(AMZN_puts["Strike"]>3950) & (AMZN_puts["Strike"]<5000)].reset_index(drop=True)
#AMZN_ATM_put

AMZN_ATM_puts=AMZN_ATM_puts.sort_values("Strike",ascending=True).reset_index(drop=True)
AMZN_ATM_puts

,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility
0,AMZN210326P03960000,2021-03-26,put,3960.0,0.00,0.00,877.28,0.000010
1,AMZN210326P03980000,2021-03-26,put,3980.0,0.00,0.00,901.65,0.000010
2,AMZN210326P03995000,2021-03-26,put,3995.0,0.00,0.00,970.94,0.000010
3,AMZN210416P04000000,2021-04-16,put,4000.0,0.00,0.00,964.27,0.000010
4,AMZN210521P04000000,2021-05-21,put,4000.0,0.00,0.00,945.64,0.000010
5,AMZN210326P04000000,2021-03-26,put,4000.0,0.00,0.00,902.80,0.000010
6,AMZN210416P04050000,2021-04-16,put,4050.0,0.00,0.00,1026.00,0.000010
7,AMZN210326P04050000,2021-03-26,put,4050.0,0.00,0.00,988.60,0.000010
8,AMZN210326P04100000,2021-03-26,put,4100.0,0.00,0.00,1047.55,0.000010
9,AMZN210521P04100000,2021-05-21,put,4100.0,1023.05,1038.00,873.70,0.631603


### Blackscholes

In [30]:
## Blackscholes function to calulate option price

# S= Stock Price

# K= Strike Price

# t= Expiration Date

# sig= Volatility

# optype= Type

# r= risk free interest rate



def blackscholes(S,K,t,optype,sig,r=0.0030):
    
    d1= (np.log(S/K)+(r+sig**2/2)*t)/(sig*np.sqrt(t))
    
    d2= d1-sig*np.sqrt(t)
        
    call_price=norm.cdf(d1,0,1)*S-  norm.cdf(d2,0,1)*K*np.exp(-r*t)
           
    put_price = K* np.exp(-r*t)* norm.cdf(-d2,0,1) -  S* norm.cdf(-d1,0,1)
    
    if optype== "call":
        
        return call_price
    
    elif optype=="put":
        
        return put_price


### Bisection

In [31]:
# bisection function compatible with apply function

def bisection(row):
    
    S=3049
    K=row["Strike"]
    optype=row["Type"]
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   
    avr_price=(row["Bid"]+row["Ask"])/2
    
    
    
    a= 0.01
    b=1
    
         
    f_b=blackscholes(S,K,t,optype,b)-avr_price
        
    f_a=blackscholes(S,K,t,optype,a)-avr_price
    
    count=0
    
    
    
    while b-a>0.01:
            
            
            count+=1
            
            if count>1000:
                
                break
            
            
                      
            
            
            c=a+b/2
            
            f_c=blackscholes(S,K,t,optype,c)-avr_price
            
            f_b=f_b
            f_a=f_a
            
            
            #f_b=blackscholes(S,K,t,optype,b)-avr_price
        
            #f_a=blackscholes(S,K,t,optype,a)-avr_price
        
            
            
            if f_c<0.01:
                
                                
                break
            
            
            if f_c*f_b<0:
                
                a=c
                
                            
            elif f_c*f_a<0:
                                
                b=c
                                
    return c

In [32]:
# example using bisection with apply function on ATM calls

AMZN_vol=AMZN_ATM_calls.apply(lambda row: bisection(row),axis=1)
AMZN_ATM_calls["bisection_implied"]=AMZN_vol
AMZN_ATM_calls


,Option Name,Expiration Date,Type,Strike,Bid,Ask,Last Price,Implied Volatility,bisection_implied
0,AMZN210326C01710000,2021-03-26,call,1710.0,0.00,0.00,1335.65,0.000010,0.510000
1,AMZN210416C01710000,2021-04-16,call,1710.0,0.00,0.00,1366.30,0.000010,0.510000
2,AMZN210416C01720000,2021-04-16,call,1720.0,0.00,0.00,1386.00,0.000010,0.510000
3,AMZN210326C01730000,2021-03-26,call,1730.0,0.00,0.00,1315.70,0.000010,0.510000
4,AMZN210416C01730000,2021-04-16,call,1730.0,0.00,0.00,1384.70,0.000010,0.510000
5,AMZN210326C01740000,2021-03-26,call,1740.0,0.00,0.00,1343.95,0.000010,0.510000
6,AMZN210416C01750000,2021-04-16,call,1750.0,0.00,0.00,1354.80,0.000010,0.510000
7,AMZN210326C01760000,2021-03-26,call,1760.0,0.00,0.00,1285.75,0.000010,0.510000
8,AMZN210416C01760000,2021-04-16,call,1760.0,0.00,0.00,1500.75,0.000010,0.510000
9,AMZN210326C01770000,2021-03-26,call,1770.0,0.00,0.00,1314.00,0.000010,0.510000


## Applying Explicit Finite Difference on AMZN

In [33]:
def getpricesEXPut(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDExplicitEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=False)
    return option.price()

In [34]:
def getpricesEXCall(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDExplicitEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=True)
    return option.price()

### Applying Explicit Finite Difference on AMZN Calls

In [35]:
EXCall=AMZN_ATM_calls.apply(getpricesEXCall, axis=1)
EXCall=pd.DataFrame(EXCall,columns=["Explicit Finite-Price"])
EXCall


,Explicit Finite-Price
0,1.315389e+03
1,1.373144e+03
2,1.369803e+03
3,1.302724e+03
4,1.366462e+03
5,1.296391e+03
6,1.359781e+03
7,1.283726e+03
8,1.356440e+03
9,1.277393e+03


### Applying Explicit Finite Difference on AMZN Puts

In [36]:
EXPut=AMZN_ATM_puts.apply(getpricesEXPut, axis=1)
EXPut=pd.DataFrame(EXPut,columns=["Explicit Finite-Price"])
EXPut

,Explicit Finite-Price
0,1.655705e+02
1,1.774905e+02
2,1.864306e+02
3,-1.538743e+02
4,-3.208825e+01
5,1.894106e+02
6,-1.526579e+02
7,2.192107e+02
8,2.490109e+02
9,1.082575e+07


## Applying Implicit Finite Difference on AMZN 

In [37]:
def getpricesIMCall(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDImplicitEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=True)
    return option.price()

In [38]:
def getpricesIMPut(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDImplicitEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=False)
    return option.price()

### Applying Implicit Finite Difference on AMZN Call

In [39]:
IMCall=AMZN_ATM_calls.apply(getpricesIMCall, axis=1)
IMCall=pd.DataFrame(IMCall,columns=["Implicit-Finite-Price"])
IMCall

,Implicit-Finite-Price
0,1229.121317
1,395.008569
2,393.734795
3,1210.816928
4,392.461022
5,1201.664733
6,389.913475
7,1183.360343
8,388.639701
9,1174.208148


### Applying Implicit Finite Difference on AMZN Puts

In [40]:
IMPut=AMZN_ATM_puts.apply(getpricesIMPut, axis=1)
IMPut=pd.DataFrame(IMPut,columns=["Implicit-Finite-Price"])
IMPut

,Implicit-Finite-Price
0,843.810805
1,861.933454
2,875.525441
3,-114.755226
4,-120.699966
5,880.056103
6,-107.996249
7,925.362727
8,970.669350
9,-5.299536


## Applying Crank-Nicolson Finite Difference on AMZN 

In [41]:
def getpricesCNCall(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDCnEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=True)
    return option.price()

In [42]:
def getpricesCNPut(row):
    
    today = datetime.today()
    exp=datetime.strptime(row["Expiration Date"],"%Y-%m-%d")
    t=(exp-today).days   

    
    option = FDCnEu(S0=3000, K=row['Strike'], r=0.06, T=t, sigma=row["Implied Volatility"], M=3, N=3, is_call=False)
    return option.price()

### Applying Crank-Nicolson Finite Difference on AMZN Call

In [43]:
CNCall=AMZN_ATM_calls.apply(getpricesCNCall, axis=1)
CNCall=pd.DataFrame(CNCall,columns=["Crack-Nicolsan-Price"])
CNCall

,Crack-Nicolsan-Price
0,1229.006953
1,282.008275
2,281.964577
3,1210.719243
4,281.920879
5,1201.575388
6,281.833483
7,1183.287679
8,281.789785
9,1174.143824


### Applying Crank-Nicolson Finite Difference on AMZN Put

In [44]:
CNPut=AMZN_ATM_puts.apply(getpricesCNPut, axis=1)
CNPut=pd.DataFrame(CNPut,columns=["Crack-Nicolsan-Price"])
CNPut

,Crack-Nicolsan-Price
0,828.360379
1,846.648089
2,860.363871
3,-272.001425
4,-59.393248
5,864.935798
6,-271.782935
7,910.655073
8,956.374347
9,604.409366


## Comparision of Call Prices

In [45]:
Call_Pricing = pd.concat([EXCall,IMCall,CNCall],axis=1)
Call_Pricing

,Explicit Finite-Price,Implicit-Finite-Price,Crack-Nicolsan-Price
0,1.315389e+03,1229.121317,1229.006953
1,1.373144e+03,395.008569,282.008275
2,1.369803e+03,393.734795,281.964577
3,1.302724e+03,1210.816928,1210.719243
4,1.366462e+03,392.461022,281.920879
5,1.296391e+03,1201.664733,1201.575388
6,1.359781e+03,389.913475,281.833483
7,1.283726e+03,1183.360343,1183.287679
8,1.356440e+03,388.639701,281.789785
9,1.277393e+03,1174.208148,1174.143824


## Comparision of Put Prices

In [46]:
Put_Pricing = pd.concat([EXPut,IMPut,CNPut],axis=1)
Put_Pricing

,Explicit Finite-Price,Implicit-Finite-Price,Crack-Nicolsan-Price
0,1.655705e+02,843.810805,828.360379
1,1.774905e+02,861.933454,846.648089
2,1.864306e+02,875.525441,860.363871
3,-1.538743e+02,-114.755226,-272.001425
4,-3.208825e+01,-120.699966,-59.393248
5,1.894106e+02,880.056103,864.935798
6,-1.526579e+02,-107.996249,-271.782935
7,2.192107e+02,925.362727,910.655073
8,2.490109e+02,970.669350,956.374347
9,1.082575e+07,-5.299536,604.409366


- We see that prices of both Calls and Puts are similar with using three different method.
- There are some difference in the prices possibly due to raw data inaccuaracy